# Assignment 6

Jonathan Farrell Kusuma

For this assignment, I will be visualizing the buildings dataset and produce two data visualizations from the agency name, county, year acquired and square footage. This visualization is going to use the bqplot library to produce interactive plots.

In [1]:
import bqplot
import numpy as np
import ipywidgets
import pandas as pd

In [2]:
buildings = pd.read_csv("https://uiuc-ischool-dataviz.github.io/is445_AOUAOG_fall2021/week02/data/building_inventory.csv",
           na_values = {'Agency Name': 0,
                       'County': 0,
                       'Square Footage': 0,
                       'Year Acquired': 0})

In [3]:
buildings

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975.0,1975,144.0,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004,144.0,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004,144.0,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004,144.0,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004,144.0,1,1,0,Unusual,Unusual,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,NaN,0,432.0,1,0,0,Storage,NaN,NaN
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,NaN,0,330.0,1,0,0,Storage,NaN,NaN
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,In Use,NaN,1987,130.0,1,0,0,Storage,High Hazard,NaN
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,In Use,1971.0,1971,49552.0,1,1,0,Education,Education,Not provided


I will be aggregating the buildings dataset using groupby. I will groupby county and agency name to find the mean of the total square footage of the buildings. Afterwards, I unstacked the dataset to produce a ndarray where the agency names are the row indexes and county as the column values using level as county.

In [4]:
aggregated_buildings = buildings.groupby(["County", "Agency Name"])["Square Footage"].mean()
aggregated_buildings = aggregated_buildings.unstack(level="County")
dept = aggregated_buildings.index
county = aggregated_buildings.columns.values
aggregated_buildings = aggregated_buildings.to_numpy()

For my first attempt in visualizing the grid heat map, I decided to follow the examples found in class notebooks and edited the names to match with the dataset. Upon looking at the results, I found that the X ticks are overlapping with each other and the colors were too similar to each other. I believe that the data has very large and small values, thus there seems to be a lot of outliers.

In [5]:
# (1) add scales - colors, x & y
col_sc = bqplot.ColorScale()
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# (2) create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')

x_ax = bqplot.Axis(scale = x_sc, label='County')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', 
                   label = 'Department')

# (3) Marks
heat_map = bqplot.GridHeatMap(color = aggregated_buildings,
                              row = dept, 
                              column = county,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})

# (4) interactivity - none yet
def on_selected(change):
    if len(change['owner'].selected) == 1:
        print(change['owner'].selected[0])
        
heat_map.observe(on_selected, 'selected')
# (5) put it all together in a figure
fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])
fig

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(), side='right'), Axis(label='Department', ori…

For the next visualization, I decided to implement the interactions, bar graph and fix the problems above. For the grid heat map, I decided to use a logarithmic scale for the colors to better show the differences in the data. Then I used a tick_rotate (source: https://bqplot.readthedocs.io/en/latest/_generate/bqplot.axes.Axis.html) to better show the X ticks. I also decided to fix the label position by giving an offset (label_offset). 

In [14]:
# (1) add scales - colors, x & y
col_sc = bqplot.ColorScale(scheme="RdPu")
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# (2) create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')

x_ax = bqplot.Axis(scale = x_sc, label='County', tick_rotate=-90, label_offset='50')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', 
                   label = 'Department')

# (3) Marks
heat_map = bqplot.GridHeatMap(color = np.log10(aggregated_buildings),
                              row = dept, 
                              column = county,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})


This section is for histogram. I decided to use (15,15) because it a lot of data for visualization. Then I filtered the buildings dataset to find the selected agency name and county. I decided to use year acquired for the year field because I think the data would be more relevant in identifying the growth of one department and county. Lasly, I decided to use a bin of 10 because it was able to efficiently show the data's trends.

In [15]:
x_scl = bqplot.LinearScale() 
y_scl = bqplot.LinearScale()
ax_xcl = bqplot.Axis(label='Year Acquired', scale=x_scl)
ax_ycl = bqplot.Axis(label='Total Square Footage', scale=y_scl, 
                    orientation='vertical', side='left')

i,j = 15, 15
masked_data = buildings[(buildings['Agency Name'] == dept[i]) & (buildings['County'] == county[j])]

In [16]:
sqft, sqft_edges = np.histogram(masked_data['Year Acquired'], 
                                weights = masked_data['Square Footage'],
                               bins = 10)

In [17]:
sqft_centers = (sqft_edges[:-1] + sqft_edges[1:]) / 2

In [18]:
sqft_hist =  bqplot.Bars(x=sqft_centers, y=sqft, 
                          scales={'x':x_scl, 'y':y_scl})

This section is for the interaction of the plots. Using if conditions, I am able to ensure the data exists (not Nan) and only one is selected. Any changes in selection will be observed and update the plot. It will also update the masked data to match with the agency name and county name.

In [19]:
def get_data_value(change):
    if len(change['owner'].selected) == 1: #only 1 selected
        i,j = change['owner'].selected[0]
        masked_data = buildings[(buildings['Agency Name'] == dept[i]) & (buildings['County'] == county[j])]
        
        if len(masked_data['Year Acquired']) > 0: # make sure point exist
            sqft, sqft_edges = np.histogram(masked_data['Year Acquired'], 
                               weights = masked_data['Square Footage'],
                               bins = 10)
            sqft_centers = (sqft_edges[:-1] + sqft_edges[1:]) / 2
            sqft_hist.x = sqft_centers
            sqft_hist.y = sqft
   
heat_map.observe(get_data_value, 'selected')

I noticed that the y ticks were cut out due to the small default margins of the visualization so I decided to use fig_margin to control the margins of the figure. (source: https://bqplot.readthedocs.io/en/latest/_generate/bqplot.figure.Figure.html). I also changed the heights and widths to make sure it is easier to read. I also decided to arrange in a vertical box since most data is landscape shaped/long. Despite changing the margins, the X ticks seems to be overlapping with the graph. After researching about this problem, I tried using a scale offset and changing the margins but did not appear to work here. So if I had more time, I would experiment more in finding how to fix this aesthetic issue.

In [20]:
fig_heatmap = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax], fig_margin={'top':80, 'bottom':200, 'left':200, 'right':100})
fig_dur = bqplot.Figure(marks = [sqft_hist], axes = [ax_xcl, ax_ycl])

In [21]:
fig_heatmap.layout.width = '1500px'
fig_heatmap.layout.height = '700px'
fig_dur.layout.min_width = '1000px'

myDashboard = ipywidgets.VBox([fig_heatmap,fig_dur])
myDashboard

In conclusion, in this visualization, I did not decide to keep the x and y ranges static because a lot of the data is too varied. Some plots has years ranging from the 1800s and some only on the 2000s; this also applies in the total square footage of the data.